# Week 6 Homework

***Due (pushed to your GitHub branch) on 10/25 by 11:59 pm***

## STAR two-pass alignment

Build the STAR SA:

In [1]:
# !mkdir placenta_alignment
# !mkdir placenta_alignment/star
# !mkdir placenta_alignment/hisat
!STAR \
--runThreadN 32 \
--runMode genomeGenerate \
--genomeDir ../2_genome_exploration/placenta_genome/star \
--genomeFastaFiles ../2_genome_exploration/placenta_genome/placenta_genome.fa \
--sjdbGTFfile ../2_genome_exploration/placenta_genome/placenta_annotations.gtf \
--sjdbOverhang 190\
--genomeSAindexNbases 13

mkdir: cannot create directory ‘placenta_alignment’: File exists
mkdir: cannot create directory ‘placenta_alignment/star’: File exists
mkdir: cannot create directory ‘placenta_alignment/hisat’: File exists
	STAR --runThreadN 32 --runMode genomeGenerate --genomeDir ../2_genome_exploration/placenta_genome/star --genomeFastaFiles ../2_genome_exploration/placenta_genome/placenta_genome.fa --sjdbGTFfile ../2_genome_exploration/placenta_genome/placenta_annotations.gtf --sjdbOverhang 190 --genomeSAindexNbases 13
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Nov 14 16:09:07 ..... started STAR run
Nov 14 16:09:07 ... starting to generate Genome files
Nov 14 16:09:47 ..... processing annotations GTF
Nov 14 16:10:10 ... starting to sort Suffix Array. This may take a long time...
Nov 14 16:10:26 ... sorting Suffix Array chunks and saving them to disk...
Nov 14 16:25:16 ... loading chunks from disk, packing SA...
Nov 14 16:26:32 ... 

Use STAR and ***all*** of your FASTQs for a first-pass alignment. Be sure to update the `sjdbOverhang` parameter to a value that is appropriate for your dataset. Rename the log file so that you can compare the logs from the first and second alignments:

In [ ]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/placenta_genome/star \
--readFilesManifest placenta_manifest.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outFileNamePrefix /placenta_alignment/star/

Use STAR, all of your FASTQs, and `SJ.out.tab` for the second-pass alignment. You will likely need to make your own `manifest.tsv`:

In [ ]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/placenta_genome/star \
--readFilesManifest placenta_manifest.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outSAMattributes NH HI AS nM RG \
--outFileNamePrefix placenta_alignment/star/ \
--sjdbFileChrStartEnd placenta_alignment/star/SJ.out.tab

Update this block to briefly describe the differences between the logs for the first and second alignments.

## HISAT alignment

(***Note***: you may need Dr. Wheeler to build the HISAT index of your genome. Ask him for assistance when you get to this part of the assignment.)

Use a Bash `while` loop to align all of your FASTQs and save the output files using the same names as the input FASTQ:

In [ ]:
!while read -r line; do \
    fq=$(echo $line | awk '{print $1}'); \
    bn=$(basename "$fq" .fastq.gz); \
    rg=$(echo $line | awk '{print $3}' | sed 's/ID://' ); \
    echo "Aligning $fq with $rg"; \
    hisat2 ../2_genome_exploration/placenta_genome/hisat/genome -p 16 -U $fq --rg-id $rg --rg SM:$rg --summary-file placenta_alignment/hisat/$bn.log --new-summary > placenta_alignment/hisat/$bn.sam; \
    done < placenta_manifest.tsv

## Snakemake Pipeline

In contrast to last week (FASTQ files), there aren't as many decisions that need to be made during alignment, so using a notebook to control and report code isn't as necessary. As you saw in in [6_alignment](6_alignment.ipynb), file and directory management can get complicated during alignment. This is a case where Snakemake scripting shines.

1. Copy your `Snakefile` and `config.yaml` from `5_fastq` to `6_alignment`. 
2. In the `Snakefile`, add the following rules:  
    a. `fetch_genome` - using a variation from the code from `2_genome_exploration`, download your reference genome and annotations.  
    b. `star_sa` - use `STAR` to produce the suffix array of the reference genome.  
    c. `star_align_first` - use `STAR` to perform first-pass alignment with all FASTQ files.  
    d. `star_align_second` - use `STAR` to perform second-pass alignment with all FASTQ files.  
    e. `hisat_align` - use `HISAT` to align all FASTQ files.  
    f. `report` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.

In [ ]:

# !STAR \
# --runThreadN 16 \
# --runMode genomeGenerate \
#     #runMode can use alignment later
# --genomeDir /data/users/wheelenj/BIOL343/2_genome_exploration/genome/star \
# --genomeFastaFiles /data/users/wheelenj/BIOL343/2_genome_exploration/genome/genome.fa \
# --sjdbGTFfile /data/users/wheelenj/BIOL343/2_genome_exploration/genome/annotations.gtf \
# --sjdbOverhang 68 \
#     #needs to be adjusted to the length of our genes
# --genomeSAindexNbases 13